In [3]:
import pandas as pd
import yfinance as yf

In [7]:
import yahooquery as yq

In [ ]:
def SP500_stocks_string(URL: str) -> str:

    tickers = pd.read_html(URL)[0]['Symbol'].tolist()

    tickers_string = ' '.join(tickers).lower()

    return tickers_string

URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

tickers_string = SP500_stocks_string(URL)

tickers_string


In [33]:
from yahooquery import Ticker

def get_price_history(ticker_string: str, period: str, ) -> pd.DataFrame:
    
    data = Ticker(ticker_string)

    df = data.history(period = period) 

    return df 

In [36]:
df = get_price_history(tickers_string, '1y')

In [118]:
def calculate_MAs(df: pd.DataFrame) -> pd.DataFrame:

    df['sma10'] = df['adjclose'].rolling(10).mean()
    df['sma30'] = df['adjclose'].rolling(30).mean()
    df['sma200'] = df['adjclose'].rolling(100).mean()

    return df

In [40]:
test_df = df

In [120]:
ma_df = calculate_MAs(test_df)

In [121]:

for symbol, new_df in ma_df.groupby(level=0):

    new_df = new_df.sort_values('date').tail(5)

In [89]:
def _calculate_pc_ma(row):
    return ((row['adjclose'] - row['sma30'])/row['sma30']) * 100

In [90]:
def _calculate_pc_ma_difference(row):
    return ((row['sma30'] - row['sma200'])/row['sma200']) * 100

In [125]:
def _calculate_pc_ma(name_of_ma_row, name_of_adj_close_row):
    return ((name_of_adj_close_row - name_of_ma_row)/name_of_ma_row) * 100

In [126]:
def calculate_ma_indicators(df: pd.DataFrame) -> pd.DataFrame:
    
    individual_df_list = []

    for symbol, new_df in df.groupby(level=0):

        new_df = new_df.sort_values('date').tail(5)

        new_df['pc_sma10_by_close'] = new_df.apply(lambda x: _calculate_pc_ma(x['sma10'], x['adjclose']), axis=1)

        new_df['pc_sma30_by_close'] = new_df.apply(lambda x: _calculate_pc_ma(x['sma30'], x['adjclose']), axis=1)
        new_df['pc_sma30_by_sma200'] = new_df.apply(_calculate_pc_ma_difference, axis=1)
        new_df['mean_of_sma30_by_close_last_5_days'] = new_df['pc_sma30_by_close'].mean()

        individual_df_list.append(new_df)

    joined_df = pd.concat(individual_df_list)

    return joined_df

In [127]:
joined_df = calculate_ma_indicators(ma_df)

In [129]:



joined_df.sort_values('pc_sma30_by_close', ascending=False).head(30)

open        high         low       close     volume  \
symbol date                                                                    
tsla   2023-02-15  211.759995  214.660004  206.110001  214.240005  182108600   
ctlt   2023-02-16   67.720001   71.970001   67.500000   71.889999    1775000   
tsla   2023-02-17  199.990005  208.440002  197.500000  208.309998  213080200   
       2023-02-16  210.779999  217.649994  201.839996  202.039993  229586500   
ctlt   2023-02-17   71.209999   71.750000   70.260002   71.370003    1818000   
       2023-02-15   69.410004   70.190002   68.559998   69.089996    1348300   
mpwr   2023-02-15  516.119995  530.650024  514.070007  530.090027     453800   
algn   2023-02-15  327.980011  339.100006  327.700012  338.390015     961200   
wst    2023-02-16  312.579987  329.070007  311.989990  319.769989    1566600   
ftnt   2023-02-15   60.990002   63.020000   60.709999   62.810001    5235500   
nvda   2023-02-15  225.490005  228.550003  221.059998  227.639999   42118800   
on     2023-02-15   85.669998   87.019997   85.459999   86.709999    6711500   
tsla   2023-02-22  197.929993  201.990005  191.779999  200.860001  191216800   
wst    2023-02-17  317.179993  319.290009  307.410004  313.720001    1040400   
ctlt   2023-02-21   69.839996   70.129997   68.370003   68.660004    1271300   
tsla   2023-02-21  204.990005  209.710007  197.220001  197.369995  180018600   
mpwr   2023-02-16  518.309998  524.869995  510.500000  513.369995     484100   
gnrc   2023-02-15  127.550003  141.539993  127.550003  135.229996    4625500   
algn   2023-02-16  331.679993  336.510010  323.730011  326.170013     983600   
nclh   2023-02-15   17.379999   18.120001   17.209999   18.110001   13260500   
rcl    2023-02-15   75.000000   76.000000   73.860001   75.750000    2611300   
ftnt   2023-02-16   62.000000   62.610001   61.230000   61.389999    4033900   
meta   2023-02-15  176.419998  178.190002  175.330002  177.160004   25338000   
ctlt   2023-02-22   68.750000   69.070000   66.889999   67.660004    1608100   
wst    2023-02-21  309.410004  309.410004  304.040009  306.679993     794500   
anet   2023-02-15  142.940002  145.169998  140.750000  141.529999    2868600   
nvda   2023-02-16  221.330002  225.500000  219.270004  220.020004   41202600   
on     2023-02-16   85.120003   86.680000   83.879997   84.129997    5871800   
mpwr   2023-02-17  512.130005  514.049988  498.510010  502.089996     488900   
aptv   2023-02-15  122.089996  124.879997  122.000000  123.070000    2731100   

                     adjclose  dividends  splits       sma30      sma200  \
symbol date                                                                
tsla   2023-02-15  214.240005        0.0     0.0  158.356000  184.429100   
ctlt   2023-02-16   71.889999        0.0     0.0   56.629667   55.859100   
tsla   2023-02-17  208.309998        0.0     0.0  164.568333  183.019200   
       2023-02-16  202.039993        0.0     0.0  161.302666  183.696200   
ctlt   2023-02-17   71.370003        0.0     0.0   57.469667   55.825500   
       2023-02-15   69.089996        0.0     0.0   55.781667   55.900900   
mpwr   2023-02-15  530.090027        0.0     0.0  430.241000  380.470900   
algn   2023-02-15  338.390015        0.0     0.0  275.693669  223.801301   
wst    2023-02-16  319.769989        0.0     0.0  263.020326  245.251525   
ftnt   2023-02-15   62.810001        0.0     0.0   52.470000   51.998900   
nvda   2023-02-15  227.639999        0.0     0.0  190.498332  158.381010   
on     2023-02-15   86.709999        0.0     0.0   73.046667   68.360800   
tsla   2023-02-22  200.860001        0.0     0.0  170.081666  181.294000   
wst    2023-02-17  313.720001        0.0     0.0  265.693264  245.864368   
ctlt   2023-02-21   68.660004        0.0     0.0   58.169667   55.784700   
tsla   2023-02-21  197.369995        0.0     0.0  167.378666  182.163500   
mpwr   2023-02-16  513.369995        0.0     0.0  435.425667  381.907707   
gnrc   2023-02-15  1

In [117]:
def latest_date_df(df: pd.DataFrame) -> pd.DataFrame): 

    df_list = []

    for symbol, new_df in joined_df.groupby(level=0):

        new_df = new_df.sort_values('date').tail(1)

        df_list.append(new_df)

    latest_date_df = pd.concat(df_list)

    latest_date_df = latest_date_df.sort_values('mean_of_sma30_by_close_last_5_days', ascending=False).head(30)

    return latest_date_df


,,open,high,low,close,volume,adjclose,dividends,splits,sma30,sma200,pc_sma30_by_close,pc_sma30_by_sma200,mean_of_sma30_by_close_last_5_days
symbol,date,,,,,,,,,,,,,
tsla,2023-02-22,197.929993,201.990005,191.779999,200.860001,191216800,200.860001,0.0,0.0,170.081666,222.654034,18.096209,-23.611684,24.627883
ctlt,2023-02-22,68.750000,69.070000,66.889999,67.660004,1608100,67.660004,0.0,0.0,58.819000,78.516900,15.030863,-25.087465,21.611528
wst,2023-02-22,306.290009,308.429993,303.790009,305.549988,558900,305.549988,0.0,0.0,270.337944,275.264011,13.025195,-1.789579,14.850328
ftnt,2023-02-22,60.599998,61.250000,59.770000,60.209999,5662100,60.209999,0.0,0.0,54.073333,53.809360,11.348785,0.490571,14.385550
mpwr,2023-02-22,485.000000,489.369995,474.739990,482.709991,383800,482.709991,0.0,0.0,447.717334,410.247460,7.815792,9.133481,14.264510
algn,2023-02-22,303.809998,308.429993,300.649994,302.720001,960500,302.720001,0.0,0.0,287.584002,243.737100,5.263158,17.989425,12.740060
meta,2023-02-22,171.070007,172.759995,169.690002,171.119995,20648300,171.119995,0.0,0.0,159.087999,150.616550,7.563107,5.624514,10.860472
cinf,2023-02-22,124.660004,125.720001,124.040001,124.709999,534500,124.709999,0.0,0.0,115.075001,107.617504,8.372799,6.929632,10.452701
on,2023-02-22,79.449997,79.910004,77.680000,78.559998,5664500,78.559998,0.0,0.0,75.539000,65.230850,3.999256,15.802568,10.438732


In [119]:
joined_df.sort_values('pc_sma30_by_close', ascending=False).head(30)

open        high         low       close     volume  \
symbol date                                                                    
tsla   2023-02-15  211.759995  214.660004  206.110001  214.240005  182108600   
ctlt   2023-02-16   67.720001   71.970001   67.500000   71.889999    1775000   
tsla   2023-02-17  199.990005  208.440002  197.500000  208.309998  213080200   
       2023-02-16  210.779999  217.649994  201.839996  202.039993  229586500   
ctlt   2023-02-17   71.209999   71.750000   70.260002   71.370003    1818000   
       2023-02-15   69.410004   70.190002   68.559998   69.089996    1348300   
mpwr   2023-02-15  516.119995  530.650024  514.070007  530.090027     453800   
algn   2023-02-15  327.980011  339.100006  327.700012  338.390015     961200   
wst    2023-02-16  312.579987  329.070007  311.989990  319.769989    1566600   
ftnt   2023-02-15   60.990002   63.020000   60.709999   62.810001    5235500   
nvda   2023-02-15  225.490005  228.550003  221.059998  227.639999   42118800   
on     2023-02-15   85.669998   87.019997   85.459999   86.709999    6711500   
tsla   2023-02-22  197.929993  201.990005  191.779999  200.860001  191216800   
wst    2023-02-17  317.179993  319.290009  307.410004  313.720001    1040400   
ctlt   2023-02-21   69.839996   70.129997   68.370003   68.660004    1271300   
tsla   2023-02-21  204.990005  209.710007  197.220001  197.369995  180018600   
mpwr   2023-02-16  518.309998  524.869995  510.500000  513.369995     484100   
gnrc   2023-02-15  127.550003  141.539993  127.550003  135.229996    4625500   
algn   2023-02-16  331.679993  336.510010  323.730011  326.170013     983600   
nclh   2023-02-15   17.379999   18.120001   17.209999   18.110001   13260500   
rcl    2023-02-15   75.000000   76.000000   73.860001   75.750000    2611300   
ftnt   2023-02-16   62.000000   62.610001   61.230000   61.389999    4033900   
meta   2023-02-15  176.419998  178.190002  175.330002  177.160004   25338000   
ctlt   2023-02-22   68.750000   69.070000   66.889999   67.660004    1608100   
wst    2023-02-21  309.410004  309.410004  304.040009  306.679993     794500   
anet   2023-02-15  142.940002  145.169998  140.750000  141.529999    2868600   
nvda   2023-02-16  221.330002  225.500000  219.270004  220.020004   41202600   
on     2023-02-16   85.120003   86.680000   83.879997   84.129997    5871800   
mpwr   2023-02-17  512.130005  514.049988  498.510010  502.089996     488900   
aptv   2023-02-15  122.089996  124.879997  122.000000  123.070000    2731100   

                     adjclose  dividends  splits       sma30      sma200  \
symbol date                                                                
tsla   2023-02-15  214.240005        0.0     0.0  158.356000  224.674617   
ctlt   2023-02-16   71.889999        0.0     0.0   56.629667   79.001850   
tsla   2023-02-17  208.309998        0.0     0.0  164.568333  223.706050   
       2023-02-16  202.039993        0.0     0.0  161.302666  224.179917   
ctlt   2023-02-17   71.370003        0.0     0.0   57.469667   78.858950   
       2023-02-15   69.089996        0.0     0.0   55.781667   79.093000   
mpwr   2023-02-15  530.090027        0.0     0.0  430.241000  409.290107   
algn   2023-02-15  338.390015        0.0     0.0  275.693669  243.511650   
wst    2023-02-16  319.769989        0.0     0.0  263.020326  275.322291   
ftnt   2023-02-15   62.810001        0.0     0.0   52.470000   53.750050   
nvda   2023-02-15  227.639999        0.0     0.0  190.498332  162.566195   
on     2023-02-15   86.709999        0.0     0.0   73.046667   64.743100   
tsla   2023-02-22  200.860001        0.0     0.0  170.081666  222.654034   
wst    2023-02-17  313.720001        0.0     0.0  265.693264  275.361974   
ctlt   2023-02-21   68.660004        0.0     0.0   58.169667   78.681850   
tsla   2023-02-21  197.369995        0.0     0.0  167.378666  223.105200   
mpwr   2023-02-16  513.369995        0.0     0.0  435.425667  409.815051   
gnrc   2023-02-15  1